In [2]:
import os
import requests
import bs4
from bs4 import BeautifulSoup
import sys
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
chrome_options = Options()
chrome_options.add_argument("--headless") 
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.implicitly_wait(5)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 90
Current browser version is 106.0.5249.121 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


In [3]:
print(bs4.__version__)

4.9.3


Rafael Amaral de Oliveira Almeida Prado

Thiago Hampl de Pierri Rocha

Para cada restaurante:

    -Nome 
    
    -"Estrelas"(0 a 5)
    
    -Avaliações
    
    -Cozinha (steakhouse,italiano,frutos do mar,...)
    
    -Preço 
    
    -Há medidas de segurança? (div class="_1ULcQ4O5 _2TXtEj3w")
    
    -Review
    

In [103]:
def pesquisa(inicial):
    url = 'https://www.tripadvisor.com.br/RestaurantSearch-g303631-oa{}-a_date.2021__2D__05__2D__26-a_people.2-a_time.20%3A00%3A00-a_zur.2021__5F__05__5F__26-Sao_Paul.html'.format(inicial)

    resposta=requests.get(url=url)

    resposta.encoding='utf-8'
    
    
    driver.get(url)
    restaurantes=driver.find_element_by_class_name("_1kXteagE")
    
    return resposta.text,restaurantes
def processa(resposta_html,restaurantes):

    soup=BeautifulSoup(resposta_html,'html.parser')
    tag_restaurantes=soup.find('div',class_="_1kXteagE")
    lista_restaurantes=tag_restaurantes.find_all('div',class_="_1llCuDZj")

    lista_nomes=[]
    lista_estrelas=[]
    lista_avaliacoes=[]
    lista_cozinhas=[]
    lista_precos=[]
    lista_reviews=[]

    patrocinio=[]

    for res_html in lista_restaurantes:
        if res_html['data-test']!='SL_list_item':
            nome=res_html.find('a',class_='_15_ydu6b').text
            nome=nome.split('.')[1]
            lista_nomes.append (nome)

            estrela=res_html.find ('svg',class_='zWXXYhVR')['title']
            palavras=estrela.split()
            nota=palavras[0]
            nota=nota.replace('"','').replace(',','.')
            nota=float(nota)
            lista_estrelas.append(nota)


            avaliacao=res_html.find('span',class_='w726Ki5B').text
            avaliacao=int(avaliacao.split()[0].replace('.',''))
            lista_avaliacoes.append(avaliacao)

            descricao=res_html.find('div',class_='MIajtJFg _1cBs8huC _3d9EnJpt')
            cozinha_e_preco=descricao.find_all('span',class_='_1p0FLy4t')
            cozinha=cozinha_e_preco[0].text
            lista_cozinhas.append(cozinha)
            preco=cozinha_e_preco[1].text

            if len(preco)<=5:
                preco2=len(preco)
            else:
                lista1=preco.split('-')
                for i in range(len(lista1)):
                    lista1[i]=lista1[i].replace(' ','')
                    lista1[i]=len(lista1[i])
                preco2=np.mean(lista1)

            lista_precos.append(preco2)    

            reviews=res_html.find_all('div',class_='_3hu2oQ7V')
            reviews=[reviews[0].text,reviews[1].text]
            lista_reviews.append(reviews)

            patrocinio.append(0)
        else:
            patrocinio.append(1)
    
    
    
    
    

    lista_seg=[]
    for restaurante in restaurantes.find_elements_by_class_name("_1llCuDZj"):
        try:
            seguranca = restaurante.find_element_by_class_name('_1Cbxp6P3').text
            lista_seg.append(1)
        except NoSuchElementException:
            lista_seg.append(0)
    for i in range(len(patrocinio)):
        if patrocinio[i]==1:
            del lista_seg[i]
    
    
    
    return lista_nomes,lista_estrelas,lista_avaliacoes,lista_cozinhas,lista_precos,lista_seg,lista_reviews

def gera_df(lista_nomes,lista_estrelas,lista_avaliacoes,lista_cozinhas,lista_precos,lista_seg,lista_reviews):
    df=pd.DataFrame({'Nome':lista_nomes,
                        'Estrelas':lista_estrelas,
                        'Avaliações':lista_avaliacoes,
                        'Cozinha':lista_cozinhas,
                        'Preço':lista_precos,
                        'Segurança':lista_seg,
                        'Reviews':lista_reviews})
    return df

def pesquisa_estruturada(inicial):
    resposta_html,restaurantes=pesquisa(inicial)
    lista_nomes,lista_estrelas,lista_avaliacoes,lista_cozinhas,lista_precos,lista_seg,lista_reviews=processa(resposta_html,restaurantes)
    df=gera_df(lista_nomes,lista_estrelas,lista_avaliacoes,lista_cozinhas,lista_precos,lista_seg,lista_reviews)
    return df

def multi_pesquisa(lista_busca):
    lista_df=[]
    
    for inicial in lista_busca:
        df=pesquisa_estruturada(inicial)
        lista_df.append(df)
        
    df_full=pd.concat(lista_df)
    return df_full.reset_index(drop=True)

In [106]:
lista=[0,30,60,90,120,150,180,210,240]
#270 restaurantes
df=multi_pesquisa(lista)
df

,Nome,Estrelas,Avaliações,Cozinha,Preço,Segurança,Reviews
0,Coco Bambu Conceito Pátio Paulista,5.0,109,Frutos do mar,2.5,0,"[“Almoço”, “Aniversário dos meus gêmeos ♡”]"
1,Coco Bambu Conceito Vila Olímpia,5.0,563,"Brasileira, Frutos do mar",2.5,1,"[“Atendimento Kaio Marquês”, “Atendimento Kaio..."
2,HUB Food Art & Lounge,5.0,1021,"Brasileira, Internacional",4.0,1,"[“Aniversário de casamento”, “Comida impecável!”]"
3,Picanharia Dos Amigos,5.0,1003,"Steakhouse, Brasileira",2.5,1,"[“Sensacional”, “Picanharia dos Amigos - Mais ..."
4,Coco Bambu Market Place,4.5,4961,"Brasileira, Bar",2.5,1,"[“Avaliação Matheus”, “Jantar”]"
...,...,...,...,...,...,...,...
265,Museo Veronica,4.5,405,"Mediterrânea, Europeia",2.5,0,"[“Ótimo custo benefício!”, “O garçom esqueceu ..."
266,Windhuk,4.5,394,"Alemã, Europeia",2.5,0,"[“Comida alemã”, “Local pitoresco e agradável”]"
267,La Frontera,4.5,261,Argentina,2.5,0,"[“Pedido pelo Ifood que Não CHEGOU”, “Não honr..."
268,Almanara,4.5,373,"Libanesa, Oriente Médio",2.5,0,"[“Opção de rodízio”, “Delivery prato errado”]"


In [107]:
df.to_csv('restaurantes.csv'.format(df), index=False)